In [2]:
import sqlite3
import pandas as pd
import numpy as np

Upon completion of this lab, each unique team in this dataset should have a record in the MongoDB instance containing the following information:

- The name of the team
- The total number of goals scored by the team during the 2011 season
- The total number of wins the team earned during the 2011 season
- A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly)
- The team's win percentage on days where it was raining during games in the 2011 season.

# Pull in the Data we need from our SQL Database

In [3]:
conn = sqlite3.connect('database.sqlite')
c = conn.cursor()

In [4]:
c.execute('''SELECT name, type FROM sqlite_master
''').fetchall()

[('sqlite_sequence', 'table'),
 ('Matches', 'table'),
 ('sqlite_autoindex_Matches_1', 'index'),
 ('Teams_in_Matches', 'table'),
 ('Teams', 'table'),
 ('Unique_Teams', 'table'),
 ('FlatView_Chrono_TeamOrder_Reduced', 'view'),
 ('FlatView_Advanced', 'view'),
 ('FlatView', 'view')]

In [5]:
c.execute('''
SELECT * FROM Matches
LIMIT 1''').fetchall()

[(1, 'D2', 2009, '2010-04-04', 'Oberhausen', 'Kaiserslautern', 2, 1, 'H')]

#### Let's grab all matches from the 2011 season

In [6]:
c.execute('''
SELECT * FROM Matches
WHERE Season LIKE '2011'
''')

## Convert to a pd.DataFrame

In [7]:
matches_df = pd.DataFrame(c.fetchall())

matches_df.columns = [i[0] for i in c.description]

In [8]:
matches_df.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


In [9]:
matches_df.shape

(992, 9)

In [10]:
matches_df.duplicated().sum()

0

#### Let's find the winning team in each match

In [11]:
def winning_team(x):
    if x['FTR'] == 'D':
        return 'Draw'
    elif x['FTR'] == 'H':
        return x['HomeTeam']
    else:
        return x['AwayTeam']

In [12]:
matches_df['Winner'] = 0

In [13]:
matches_df.apply(winning_team, axis = 1)

0      Bayern Munich
1      Bayern Munich
2      Bayern Munich
3              Mainz
4               Draw
           ...      
987       Man United
988          Swansea
989        Tottenham
990          Arsenal
991            Wigan
Length: 992, dtype: object

In [14]:
matches_df['Winner'] = matches_df.apply(winning_team, axis = 1)

In [15]:
matches_df.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Winner
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,Bayern Munich
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,Bayern Munich
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,Bayern Munich
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,Mainz
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,Draw


### Setting up away and home win/loss/draw information to use groupby later

In [16]:
def away_wins(x):
    if x['FTR'] == 'A':
        return 1
    else:
        return 0

In [17]:
def home_wins(x):
    if x['FTR'] == 'H':
        return 1
    else:
        return 0

In [18]:
def draws(x):
    if x['FTR'] == 'D':
        return 1
    else:
        return 0

In [19]:
def away_losses(x):
    if x['FTR'] != 'A' and x['FTR'] != 'D':
        return 1
    else:
        return 0

In [20]:
def home_losses(x):
    if x['FTR'] != 'H' and x['FTR'] != 'D':
        return 1
    else:
        return 0

In [21]:
matches_df['away_wins'] = matches_df.apply(away_wins, axis = 1)

matches_df['home_wins'] = matches_df.apply(home_wins, axis = 1)

matches_df['draws'] = matches_df.apply(draws, axis = 1)

matches_df['away_losses'] = matches_df.apply(away_losses, axis = 1)

matches_df['home_losses'] = matches_df.apply(home_losses, axis = 1)

In [22]:
matches_df.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Winner,away_wins,home_wins,draws,away_losses,home_losses
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,Bayern Munich,1,0,0,0,1
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,Bayern Munich,1,0,0,0,1
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,Bayern Munich,1,0,0,0,1
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,Mainz,0,1,0,1,0
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,Draw,0,0,1,0,0


### Let's check if our groupby now works to get the total wins, draws, and losses for each team 

In [25]:
# Wins and draws at HOME per team

matches_df.groupby(['HomeTeam'])[['home_wins', 'draws', 'home_losses']].sum().loc['Arsenal', :]

home_wins      12
draws           4
home_losses     3
Name: Arsenal, dtype: int64

In [26]:
# Wins and draws AWAY per team

matches_df.groupby(['AwayTeam'])[['away_wins', 'draws', 'away_losses']].sum().loc['Arsenal', :]

away_wins      9
draws          3
away_losses    7
Name: Arsenal, dtype: int64

### We can make sure our numbers are correct. I will use Arsenal's record as a check

#### According to our groupbys Arsenal played 19 home games, and 19 away games

In [ ]:
# total home games played

matches_df.groupby(['HomeTeam'])['home_wins'].count()['Arsenal']

In [ ]:
# Total away games played

matches_df.groupby(['AwayTeam'])['away_wins'].count()['Arsenal']

#### They had 12 wins and 4 draws at home, which leaves them with a 12 - 4 - 3 record
#### They had 9 wins and 3 draws away, which leaves them with a 9-3-7 record
#### Adding these up gives a 21-7-10 record which is correct based on a Google search of Arsenal's 2011-2012 record

## Groupby to calculate home and away losses

In [ ]:
matches_df.groupby(['HomeTeam'])['home_wins'].count() - (matches_df.groupby(['HomeTeam'])['home_wins'].sum() + 
                                                         matches_df.groupby(['HomeTeam'])['draws'].sum())

In [ ]:
matches_df.groupby(['AwayTeam'])['away_wins'].count() - (matches_df.groupby(['AwayTeam'])['away_wins'].sum() + 
                                                         matches_df.groupby(['AwayTeam'])['draws'].sum())

# Grabbing data from our API. We need to add rain data before aggregating

In [27]:
import requests
import time
from datetime import datetime

Berlin lat/long 52.5200° N, 13.4050° E

In [ ]:
ds_api_key = ''

Unneeded info

currently
minutely
hourly
alerts
flags

In [ ]:
req = requests.api.get(
    f'https://api.darksky.net/forecast/{ds_api_key}/52.52,13.405,2018-05-13T10:00:00?exclude=hourly')

In [ ]:
req.json()

In [ ]:
req.json()['daily']['data'][0]['precipIntensityMax']

In [ ]:
matches_df['Date'].unique()

### Check our API request count

In [ ]:
matches_df['Date'].nunique()

### Write API Request function. We are just populating a list with precipIntensityMax for each day. If it's > 0, we will keep that as a rainy day. 0 is non-rainy

In [ ]:
def grab_precip_info(ds_api_key, date_list):
    output_list = []
    for date in date_list:
        req = requests.api.get(
            f'https://api.darksky.net/forecast/{ds_api_key}/52.52,13.405,{date}T10:00:00?exclude=hourly')
        output_list.append(req.json()['daily']['data'][0]['precipIntensityMax'])
    return output_list

#### Run our function and set variable to our list

In [ ]:
rain_on_date = grab_precip_info(ds_api_key, matches_df['Date'].unique())

### Create a rain df with unique IDs (we will join this with our main df)

In [ ]:
rain_df = pd.DataFrame(rain_on_date, matches_df['Date'].unique())

rain_df = rain_df.rename(columns = {0 : 'rainfall'})

In [ ]:
rain_df.head()

### Set our matches_df index as date and join our tables

In [ ]:
matches_df = matches_df.set_index('Date')

In [ ]:
matches_rainfall_df = matches_df.join(rain_df, on = matches_df.index)

In [ ]:
matches_rainfall_df.head()

#### Check that our merge worked properly. Every 2011-01-15 match is getting the same rainfall number

In [ ]:
matches_rainfall_df.loc['2012-03-31', :]

#### Create a boolean column for rainfall

In [ ]:
matches_rainfall_df['rainfall_bool'] = matches_rainfall_df['rainfall'] != False

In [ ]:
matches_rainfall_df.head()

#### We need the home and away wins when raining

In [ ]:
def away_wins_rain(x):
    if x['FTR'] == 'A' and x['rainfall_bool']:
        return 1
    else:
        return 0

In [ ]:
def home_wins_rain(x):
    if x['FTR'] == 'H' and x['rainfall_bool']:
        return 1
    else:
        return 0

In [ ]:
matches_rainfall_df['away_wins_rain'] = matches_rainfall_df.apply(away_wins_rain, axis = 1)

matches_rainfall_df['home_wins_rain'] = matches_rainfall_df.apply(home_wins_rain, axis = 1)

In [ ]:
matches_rainfall_df.head()

### Create home and away dfs

In [ ]:
home_df = matches_rainfall_df.groupby('HomeTeam').sum()[['home_wins', 'draws', 'home_losses', 'FTHG', 
                                               'home_wins_rain', 'rainfall_bool']]

away_df = matches_rainfall_df.groupby('AwayTeam').sum()[['away_wins', 'draws', 'away_losses', 'FTAG', 
                                               'away_wins_rain', 'rainfall_bool']]

home_df.columns = ['wins', 'draws', 'losses', 'goals', 'wins_rain', 'rainfall_bool']

away_df.columns = ['wins', 'draws', 'losses', 'goals', 'wins_rain', 'rainfall_bool']

In [ ]:
home_df.head()

In [ ]:
away_df.head()

In [ ]:
final_df = home_df + away_df

In [ ]:
final_df['rain_win_percent'] = final_df['wins_rain'] / final_df['rainfall_bool']

In [ ]:
final_df.head()

### Create "histograms" aka bar charts - I would argue that these aren't really histograms

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
win_loss_graphs = []
for count, team in enumerate(final_df.index):
    fig = plt.figure(count)
    (plt.bar(x = ['wins', 'losses'], height = [final_df.loc[team, 'wins'], final_df.loc[team, 'losses']], 
            color = ['red', 'blue']))
    win_loss_graphs.append(fig)

In [ ]:
type(win_loss_graphs[0])

#### Now we need to convert this list to binary or bson, so it can be read into a Mongo database

In [ ]:
binarized_figs = []
for item in win_loss_graphs:
    item.savefig('myfile.png')
    with open("myfile.png", "rb") as image:
        f = image.read()
    binarized_figs.append(f)

In [ ]:
final_df['graph'] = binarized_figs

In [ ]:
final_df.head()

### Reset index so the Team Name is properly stored

In [ ]:
final_df.reset_index(inplace = True)

In [ ]:
final_df.head()

### Put our records in our Mongo database with PyMongo

In [ ]:
import pymongo

In [ ]:
#Start db
myclient = pymongo.MongoClient('mongodb://127.0.0.1:27017')
mydb = myclient['team_database']
mycollection = mydb['matches_collection']

In [ ]:
#Prep records for db
final_dict = final_df.to_dict(orient = 'records')

In [ ]:
final_dict[0]

In [ ]:
#Add records to db
insertion_results = mycollection.insert_many(final_dict)
insertion_results.inserted_ids

In [ ]:
# Call db
get_db = myclient.get_database('team_database')

my_matches = get_db.get_collection('matches_collection')

In [ ]:
my_matches

In [ ]:
cursor = my_matches

rebuild_df = []
for document in cursor.find():
    rebuild_df.append(document)

In [ ]:
df_back = pd.DataFrame(rebuild_df)

In [ ]:
df_back

In [ ]:
df_back.loc[0,'graph']

In [ ]:
from PIL import Image
import io

image = Image.open(io.BytesIO(df_back.loc[0,'graph']))

In [ ]:
image

In [ ]:
myclient.drop_database(mydb)